In [1]:

############Adjustments##############

#options
PATH_DI06C001 = '/media/sf_SF/Stage2021/targetedQE/' 

## Adjustments
path = PATH_DI06C001

filename_Y_labels = 'total_y_matrix_with_binary_label.txt'

########################



# load libraries
import pandas as pd
import os
import random
from PIL import Image
import numpy as np


#set paths
path_data_in = path + 'data/input/' + 'MachineLearning/'
path_data_out = path + 'data/output/' + 'MachineLearning/'
path_data_X = path_data_in + 'Xarrays/' #png's
path_data_y = path_data_in + 'Yarrays/' #labels



## Y
#load all Y labels together
filename = path_data_y + filename_Y_labels
# print(filename)

y = pd.read_csv(filename, sep='\t')



## X
#list all X files and devide in train OR test folder
filenames_X_train = []
filenames_X_test = []
directory_list = os.listdir(path_data_X)
# print(directory_list)

#random order list with filenames
random.shuffle(directory_list)

os.chdir(path_data_X)

i = 0
for filename in directory_list:
    #print (filename) #all files, folders
    #print (i)
    if ".png" in filename :
        #print (filename)
        if i % 3 == 0: 
            #1/3th of data is test set, rest in train
            #print(i)
            filenames_X_test.append(path_data_X + filename)
        else:
            filenames_X_train.append(path_data_X + filename)
        i = i + 1
        
 #check ok? 70-30 devide train - test? ok     
print(len(filenames_X_train))
print(len(filenames_X_test))


## load X data + Merge per train/test X's with Y to S1
#keep only non unique values


def load_X_if_matched_in_y(filenames_list, y):
    all_images_as_array=[]
    label=[]  
    global ordered_filenames
    ordered_filenames=pd.DataFrame()
    # match = 0
    # no_match = 0
    for filename in filenames_list:
        #print(filename)
        #filename = filenames_X_train[3]
        filename_wopath = filename.split('Xarrays/')[1]
        #filename_wopath = filename_wopath[:-4] #wo .png todo, see same x/y !!!
        #filename_wobw = filename_wopath.split('_bw')[0]+".png"
        #print(filename_wopath)
    
        matching_y = y[y.png==filename_wopath]
        #print(matching_y)
        if len(matching_y) == 1:
            file_lab = pd.DataFrame()
            lab = matching_y.iloc[0,2] #1st elem contains string NF/FOUND
            label.append(lab) 
            
            #load figure correctly as array [[], [], []]]
            img=Image.open(filename)
            np_array = np.asarray(img)
            #print(np_array.shape)
            
            l,b,c = np_array.shape    
            np_array = np_array.reshape(l*b*c,)   
            all_images_as_array.append(np_array)
            
            file_lab = pd.DataFrame(np.array([[filename_wopath,lab]]))
            #print(file_lab)
            ordered_filenames = ordered_filenames.append(file_lab, ignore_index=True)
            
        if len(matching_y) != 1:
            # print("no or multiple match(es) in y found for: " + filename)
            # no_match = no_match + 1
            continue
            
    #print(ordered_filenames)
    return np.array(all_images_as_array), np.array(label)
    

#if re.match(filename_wopath, y.Name[0]): #todo search in volled colom, ev niet via regress want wo .png moet volled zelfde
        


X_train,y_train = load_X_if_matched_in_y(filenames_X_train, y)
X_test, y_test = load_X_if_matched_in_y(filenames_X_test, y)



3406
1704


In [2]:
from tensorflow import keras
from tensorflow.keras import layers


from kerastuner.tuners import RandomSearch
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters


In [4]:
X_train = X_train.reshape(-1,128, 128,1)
X_test = X_test.reshape(-1,128, 128,1)

# Normalisatie
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


In [6]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(128, 128)))
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), 32, 512, 32),
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model


tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='test_dir')

tuner.search_space_summary()

tuner.search(x=X_train,
             y=y_train,
             epochs=3,
             validation_data=(X_test, y_test))

tuner.results_summary()


Trial 5 Complete [00h 00m 55s]
val_accuracy: 0.6606990694999695

Best val_accuracy So Far: 0.6606990694999695
Total elapsed time: 00h 04m 07s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in test_dir/untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 18
units_0: 256
units_1: 352
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
units_17: 32
Score: 0.6606990694999695
Trial summary
Hyperparameters:
num_layers: 12
units_0: 384
units_1: 128
learning_rate: 0.01
units_2: 224
units_3: 192
units_4: 128
units_5: 256
units_6: 512
units_7: 320
units_8: 384
units_9: 320
units_10: 288
units_11: 192
units_12: 320
units_13: 64
units_14: 352
units_15: 352
units_16: 480
units_17: 320
Score: 0.6606990694999695
Trial summary
Hyperparameters:
num_layers: 7

In [7]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    loss=keras.losses.SparseCategoricalCrossentropy(name='my_loss'),
    metrics=['accuracy', 'mse'],
    max_trials=5,
    directory='test_dir')

tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

INFO:tensorflow:Reloading Oracle from existing project test_dir/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from test_dir/untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [8]:
class MyHyperModel(HyperModel):

    def __init__(self, img_size, num_classes):
        self.img_size = img_size
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Flatten(input_shape=self.img_size))
        for i in range(hp.Int('num_layers', 2, 20)):
            model.add(layers.Dense(units=hp.Int('units_' + str(i), 32, 512, 32),
                                   activation='relu'))
        model.add(layers.Dense(self.num_classes, activation='softmax'))
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
        return model


tuner = RandomSearch(
    MyHyperModel(img_size=(128, 128), num_classes=2),
    objective='val_accuracy',
    max_trials=5,
    directory='test_dir')

tuner.search(X_train,
             y=y_train,
             epochs=5,
             validation_data=(X_test, y_test))

INFO:tensorflow:Reloading Oracle from existing project test_dir/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from test_dir/untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [9]:
hp = HyperParameters()
hp.Choice('learning_rate', [1e-1, 1e-3])

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    tune_new_entries=False,
    objective='val_accuracy')

tuner.search(x=X_train,
             y=y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 2 Complete [00h 00m 05s]
val_accuracy: 0.6606990694999695

Best val_accuracy So Far: 0.6606990694999695
Total elapsed time: 00h 00m 12s
INFO:tensorflow:Oracle triggered exit


In [10]:
hp = HyperParameters()
hp.Fixed('learning_rate', 0.1)

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    tune_new_entries=True,
    objective='val_accuracy')

tuner.search(x=X_train,
             y=y_train,
             epochs=5,
             validation_data=(X_test, y_test))

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [11]:
hp = HyperParameters()
hp.Choice('learning_rate', [1e-1, 1e-3])

tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    tune_new_entries=True,
    objective='val_accuracy')

tuner.search(x=X_train,
             y=y_train,
             epochs=5,
             validation_data=(X_test, y_test))


INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json
INFO:tensorflow:Oracle triggered exit


In [12]:
hp = HyperParameters()
hp.Choice('learning_rate', [1e-1, 1e-3])
hp.Int('num_layers', 2, 20)


def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(128, 128)))
    for i in range(hp.get('num_layers')):
        model.add(layers.Dense(32,
                               activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.get('learning_rate')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model


tuner = RandomSearch(
    build_model,
    max_trials=5,
    hyperparameters=hp,
    allow_new_entries=False,
    objective='val_accuracy')

tuner.search(x=X_train,
             y=y_train,
             epochs=5,
             validation_data=(X_test, y_test))

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
Invalid model 0/5


Traceback (most recent call last):
  File "/home/bit/.local/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-12-1fed6761ad5f>", line 9, in build_model
    for i in range(hp.get('num_layers')):
  File "/home/bit/.local/lib/python3.6/site-packages/kerastuner/engine/hyperparameters.py", line 653, in get
    raise ValueError('{} does not exist.'.format(name))
ValueError: num_layers does not exist.
Traceback (most recent call last):
  File "/home/bit/.local/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-12-1fed6761ad5f>", line 9, in build_model
    for i in range(hp.get('num_layers')):
  File "/home/bit/.local/lib/python3.6/site-packages/kerastuner/engine/hyperparameters.py", line 653, in get
    raise ValueError('{} does not exist.'.format(name))
ValueError: num_layers does not exist.
Traceback (most rece

Invalid model 1/5
Invalid model 2/5
Invalid model 3/5


Traceback (most recent call last):
  File "/home/bit/.local/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-12-1fed6761ad5f>", line 9, in build_model
    for i in range(hp.get('num_layers')):
  File "/home/bit/.local/lib/python3.6/site-packages/kerastuner/engine/hyperparameters.py", line 653, in get
    raise ValueError('{} does not exist.'.format(name))
ValueError: num_layers does not exist.
Traceback (most recent call last):
  File "/home/bit/.local/lib/python3.6/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "<ipython-input-12-1fed6761ad5f>", line 9, in build_model
    for i in range(hp.get('num_layers')):
  File "/home/bit/.local/lib/python3.6/site-packages/kerastuner/engine/hyperparameters.py", line 653, in get
    raise ValueError('{} does not exist.'.format(name))
ValueError: num_layers does not exist.
Traceback (most rece

Invalid model 4/5
Invalid model 5/5


RuntimeError: Too many failed attempts to build model.